# NST for videos

In [ ]:
import numpy as np
import os
import cv2
import tensorflow as tf
from google.colab import files

!git clone https://github.com/TeamCV3/video-style-transfer -q
%cd video-style-transfer/custom_nst
uploaded_file = None
from custom_stylization_model import CustomStylizationModel

/content/video-style-transfer/video-style-transfer/custom_nst


In [ ]:
"""
This is the main function that applies our Neural Network framewise,
and concatenates resulting frames in a film.
Further, it applies optical flow between frames for less noisy picture.
"""

def style_video(video, style):

  # function, that gets n-th digit (used for naming purposes)
  def get_digit(number, n):
    return number // 10**n % 10

  # create an array of images from folder
  def get_image_array(folder):
    array = []
    for filename in sorted(os.listdir(folder)):
      img = cv2.imread(f"{folder}/{filename}")
      array.append(img)
    return array

  """
  Split target video into frames and load the frames to a folder
  """
  vidcap = cv2.VideoCapture(video)
  success,image = vidcap.read()
  count = 0
  print("Frames upload started...")
  while success:
    # used for naming
    tt = get_digit(count, 4)
    t = get_digit(count, 3)
    h = get_digit(count, 2)
    ts = get_digit(count, 1)
    o = get_digit(count, 0)
    number = str(tt) + str(t) + str(h) + str(ts) + str(o)
    # save a frame
    cv2.imwrite(f"frames/frame_{number}.jpg", image)     
    success,image = vidcap.read()
    count += 1
  print(count,"frames uploaded")

  """
  Apply style transfer frame-wise. Here, to overcome colab limitations, we use less FPS
  and less iterations of our model.
  """
  i = 0 
  file_list = sorted(os.listdir("frames"))
  print("Style started...")
  for filename in file_list:
    if i%2==0:
      # call the model
      model = CustomStylizationModel(
          f'frames/{filename}',
          style,
          content_layers=['block5_conv2'],
          style_layers=['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1'],
          style_weight=1e-3,
          content_weight=1e3*5,
          total_variation_weight=30,
          n_iter=100,
          opt=tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
      )
      result = model.compose()
      result.save(f'result_frames/{filename}')
    i+=1
    # print progress
    if i%100==0:
      print(np.round(i/len(file_list),2)*100,"% complete...")
  
  """
  Saving frames into videos
  """

  res_copy = get_image_array("result_frames")
  frames_r = get_image_array("frames")[0::2]

  height, width, layers = frames_r[0].shape
  fourcc = cv2.VideoWriter_fourcc(*'XVID')

  # standard video of styled images
  video_std_name = 'result_std.mp4'
  video_std = cv2.VideoWriter(video_std_name,fourcc, 12, (width,height))
  # video of styled images + optical flow
  video_opt_flow_name = 'result_opt_flow.mp4'
  video_opt_flow = cv2.VideoWriter(video_opt_flow_name,fourcc, 12, (width,height))
  
  # write frames to standard video
  for img in res_copy:
    video_std.write(img)
  video_std.release()
  
  """
  Optical flow part:
  1) Get original and styled images
  2) Calculate flow between original consecutive frames
  3) Transfer difference from step 2 into styles frames
  4) Save new frames in a video
  """ 
  for i in range(1, len(res_copy)):
    # load frames to compare and create a difference mask
    prev = cv2.cvtColor(frames_r[i-1], cv2.COLOR_BGR2GRAY)
    current = cv2.cvtColor(frames_r[i], cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(frames_r[i].shape)

    mask[..., 1] = 255

    # Calculates dense optical flow by Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev, current,
            None,
            0.5, 3, 13, 3, 5, 1.2, 0)
    
    # Computes the magnitude and angle of the 2D vectors
    magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    
    # magnitude (normalized)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX) / 255
    
    # get magnitute as 3-channel (for each colour)
    rgb = np.dstack([magnitude,magnitude,magnitude])
    # filter magnitute
    rgb = np.where(rgb < 0.06, rgb, 1)
    
    # leave unchanged regions from previous frame and only update changed ones from current frame
    current_w = (res_copy[i-1] * (1 - rgb) + res_copy[i] * rgb).astype(np.uint8)
    res_copy[i] = current_w
    video_opt_flow.write(current_w)
  video_opt_flow.release()

In [ ]:
video = files.upload()
video_filename = next(iter(video))

Saving video_final.mp4 to video_final.mp4


In [ ]:
style = files.upload()
style_filename = next(iter(style))

Saving style (1).png to style (1).png


In [ ]:
! rm -rf frames
! mkdir frames

! rm -rf result_frames
! mkdir result_frames

In [ ]:
style_video(video_filename, style_filename)

Frames upload started...
1646 frames uploaded
Style started...
6.0 % complete...
12.0 % complete...
18.0 % complete...
24.0 % complete...
30.0 % complete...
36.0 % complete...
43.0 % complete...
49.0 % complete...
55.00000000000001 % complete...
61.0 % complete...
